# CORN MLP for ordinal regression and deep learning -- cement strength dataset

This tutorial explains how to train a deep neural network (here: multilayer perceptron) with the CORN loss function for ordinal regression. 

**CORN reference:**

- Xintong Shi, Wenzhi Cao, and Sebastian Raschka 
[Deep Neural Networks for Rank-Consistent Ordinal Regression Based On Conditional Probabilities](https://arxiv.org/abs/2111.08851).

## 0 -- Obtaining and preparing the cement_strength dataset

We will be using the cement_strength dataset from [https://github.com/gagolews/ordinal_regression_data/blob/master/cement_strength.csv](https://github.com/gagolews/ordinal_regression_data/blob/master/cement_strength.csv).

First, we are going to download and prepare the and save it as CSV files locally. This is a general procedure that is not specific to CORN.

This dataset has 5 ordinal labels (1, 2, 3, 4, and 5). Note that CORN requires labels to be starting at 0, which is why we subtract "1" from the label column.

In [1]:
import pandas as pd
import numpy as np


data_df = pd.read_csv("https://raw.githubusercontent.com/gagolews/ordinal_regression_data/master/cement_strength.csv")
 
data_df["response"] = data_df["response"]-1 # labels should start at 0

data_labels = data_df["response"]
data_features = data_df.loc[:, ["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8"]]

print('Number of features:', data_features.shape[1])
print('Number of examples:', data_features.shape[0])
print('Labels:', np.unique(data_labels.values))

Number of features: 8
Number of examples: 998
Labels: [0 1 2 3 4]


### Split into training and test data

In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    data_features.values,
    data_labels.values,
    test_size=0.2,
    random_state=1,
    stratify=data_labels.values)

### Standardize features

In [3]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

## 1 -- Setting up the dataset and dataloader

In this section, we set up the data set and data loaders. This is a general procedure that is not specific to CORN. 

In [4]:
import torch


##########################
### SETTINGS
##########################

# Hyperparameters
random_seed = 1
learning_rate = 0.001
num_epochs = 20
batch_size = 128

# Architecture
NUM_CLASSES = 5

# Other
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Training on', DEVICE)

Training on cuda:0


In [5]:
from torch.utils.data import Dataset


class MyDataset(Dataset):

    def __init__(self, feature_array, label_array, dtype=np.float32):
    
        self.features = feature_array.astype(np.float32)
        self.labels = label_array

    def __getitem__(self, index):
        inputs = self.features[index]
        label = self.labels[index]
        return inputs, label

    def __len__(self):
        return self.labels.shape[0]

In [6]:
import torch
from torch.utils.data import DataLoader


# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = MyDataset(X_train_std, y_train)
test_dataset = MyDataset(X_test_std, y_test)


train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True, # want to shuffle the dataset
                          num_workers=0) # number processes/CPUs to use

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=0)

# Checking the dataset
for inputs, labels in train_loader:  
    print('Input batch dimensions:', inputs.shape)
    print('Input label dimensions:', labels.shape)
    break

Input batch dimensions: torch.Size([128, 8])
Input label dimensions: torch.Size([128])


## 2 - Equipping MLP with a CORN layer

In this section, we are implementing a simple MLP for ordinal regression with CORN. Note that the only specific modification required is setting the number of output of the last layer (a fully connected layer) to the number of classes - 1 (these correspond to the binary tasks used in the extended binary classification as described in the paper).

In [7]:
class MLP(torch.nn.Module):

    def __init__(self, in_features, num_classes, num_hidden_1=300, num_hidden_2=300):
        super().__init__()
        
        self.my_network = torch.nn.Sequential(
            
            # 1st hidden layer
            torch.nn.Linear(in_features, num_hidden_1, bias=False),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.BatchNorm1d(num_hidden_1),
            
            # 2nd hidden layer
            torch.nn.Linear(num_hidden_1, num_hidden_2, bias=False),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.BatchNorm1d(num_hidden_2),
            
            ### Specify CORN layer
            torch.nn.Linear(num_hidden_2, (num_classes-1))
            ###--------------------------------------------------------------------###
        )
                
    def forward(self, x):
        logits = self.my_network(x)
        return logits
    
    
    
torch.manual_seed(random_seed)
model = MLP(in_features=8, num_classes=NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 3 - Using the CORN loss for model training

During training, all you need to do is to use the `corn_loss` provided via `coral_pytorch`. The loss function will take care of the conditional training set processing and modeling the conditional probabilities used in the chain rule (aka general product rule). 

In [8]:
#!pip install coral-pytorch

In [9]:
from coral_pytorch.losses import corn_loss


for epoch in range(num_epochs):
    
    model = model.train()
    for batch_idx, (features, class_labels) in enumerate(train_loader):

        class_labels = class_labels.to(DEVICE)
        features = features.to(DEVICE)
        logits = model(features)
        
        #### CORN loss 
        loss = corn_loss(logits, class_labels, NUM_CLASSES)
        ###--------------------------------------------------------------------###   
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 200:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), loss))

Epoch: 001/020 | Batch 000/007 | Cost: 44.5581
Epoch: 002/020 | Batch 000/007 | Cost: 37.2149
Epoch: 003/020 | Batch 000/007 | Cost: 31.0465
Epoch: 004/020 | Batch 000/007 | Cost: 31.5560
Epoch: 005/020 | Batch 000/007 | Cost: 26.1428
Epoch: 006/020 | Batch 000/007 | Cost: 24.7515
Epoch: 007/020 | Batch 000/007 | Cost: 23.3453
Epoch: 008/020 | Batch 000/007 | Cost: 23.9005
Epoch: 009/020 | Batch 000/007 | Cost: 21.4760
Epoch: 010/020 | Batch 000/007 | Cost: 22.7869
Epoch: 011/020 | Batch 000/007 | Cost: 22.6931
Epoch: 012/020 | Batch 000/007 | Cost: 21.0200
Epoch: 013/020 | Batch 000/007 | Cost: 21.9232
Epoch: 014/020 | Batch 000/007 | Cost: 19.6500
Epoch: 015/020 | Batch 000/007 | Cost: 18.7777
Epoch: 016/020 | Batch 000/007 | Cost: 20.1790
Epoch: 017/020 | Batch 000/007 | Cost: 19.0979
Epoch: 018/020 | Batch 000/007 | Cost: 17.8689
Epoch: 019/020 | Batch 000/007 | Cost: 18.2110
Epoch: 020/020 | Batch 000/007 | Cost: 16.4955


## 4 -- Evaluate model

Finally, after model training, we can evaluate the performance of the model. For example, via the mean absolute error and mean squared error measures.

For this, we are going to use the `corn_label_from_logits` utility function from `coral_pytorch` to convert the probabilities back to the orginal label.


In [10]:
from coral_pytorch.dataset import corn_label_from_logits


def compute_mae_and_mse(model, data_loader, device):

    with torch.no_grad():
    
        mae, mse, acc, num_examples = 0., 0., 0., 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            predicted_labels = corn_label_from_logits(logits).float()

            num_examples += targets.size(0)
            mae += torch.sum(torch.abs(predicted_labels - targets))
            mse += torch.sum((predicted_labels - targets)**2)

        mae = mae / num_examples
        mse = mse / num_examples
        return mae, mse

In [11]:
train_mae, train_mse = compute_mae_and_mse(model, train_loader, DEVICE)
test_mae, test_mse = compute_mae_and_mse(model, test_loader, DEVICE)

In [12]:
print(f'Mean absolute error (train/test): {train_mae:.2f} | {test_mae:.2f}')
print(f'Mean squared error (train/test): {train_mse:.2f} | {test_mse:.2f}')

Mean absolute error (train/test): 0.28 | 0.34
Mean squared error (train/test): 0.34 | 0.36


Note that MNIST is not an ordinal dataset (there is no order between the image categories), so computing the MAE or MSE doesn't really make sense but we use it anyways for demonstration purposes.

## 5 -- Rank probabilities from logits

To obtain the rank probabilities from the logits, you can use the sigmoid function to get the conditional probabilities for each task and then compute the task probabilities via the chain rule for probabilities. Note that this is also done internally by the `corn_label_from_logits` we used above.

In [13]:
logits = model(features)

with torch.no_grad():
    probas = torch.sigmoid(logits)
    probas = torch.cumprod(probas, dim=1)
    print(probas)

tensor([[8.3935e-01, 1.2113e-01, 1.6136e-02, 1.1445e-02],
        [9.6112e-01, 9.5489e-01, 7.8000e-01, 4.2623e-01],
        [9.7362e-01, 9.6609e-01, 1.8540e-01, 5.9327e-02],
        [3.6196e-01, 2.1423e-03, 1.1401e-04, 5.7968e-05],
        [2.8502e-04, 2.4822e-05, 1.4027e-05, 6.4623e-06],
        [1.9014e-02, 2.5819e-06, 4.6350e-07, 4.6239e-07],
        [9.6274e-01, 2.6348e-01, 5.7620e-03, 1.9632e-03],
        [9.8527e-01, 8.7068e-01, 4.2265e-01, 3.8741e-01],
        [8.3690e-01, 2.3642e-01, 1.7974e-02, 9.9121e-04],
        [9.5583e-01, 6.6400e-01, 2.2153e-01, 9.9216e-02],
        [9.7903e-01, 2.9021e-01, 6.2193e-03, 2.3090e-04],
        [1.5357e-01, 3.8619e-04, 9.2371e-05, 8.5048e-05],
        [5.8292e-01, 1.9840e-01, 1.3502e-01, 9.6535e-02],
        [6.3317e-01, 3.7718e-03, 2.8098e-04, 2.3928e-04],
        [9.5486e-01, 9.5452e-01, 9.5238e-01, 8.8611e-01],
        [9.2712e-01, 9.2471e-01, 8.8670e-01, 5.9615e-01],
        [9.9412e-01, 3.1466e-01, 2.7411e-02, 7.9967e-04],
        [9.862